In [2]:
import pandas as pd
import numpy as np
from sklearn.svm import SVR
# from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, reciprocal
from common.preprocessor import load_data
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform, randint
%matplotlib inline

In [3]:
data = load_data('data', 'Commodity Prices Monthly.csv')
data.head()

,Price
2002-01-01,19.96
2002-02-01,20.19
2002-03-01,24.03
2002-04-01,26.03
2002-05-01,25.69


In [1]:
len(data)

NameError: name 'data' is not defined

In [10]:
# use plotly for this visualization
import plotly.express as px
fig = px.line(data.reset_index(), x='index', y='Price', title='Commodity Prices Monthly')
fig.show()

In [11]:
# set the train and test data with start dates
train_start_date = '2002-01-01'
test_start_date = '2019-01-01'

In [13]:
import plotly.graph_objects as go

# Create a trace for the train data
train_trace = go.Scatter(
    x = data[(data.index < test_start_date) & (data.index >= train_start_date)].index,
    y = data[(data.index < test_start_date) & (data.index >= train_start_date)]['Price'],
    mode = 'lines',
    name = 'train'
)

# Create a trace for the test data
test_trace = go.Scatter(
    x = data[test_start_date:].index,
    y = data[test_start_date:]['Price'],
    mode = 'lines',
    name = 'test'
)

# Create the layout
layout = go.Layout(
    title = 'Train and Test Data',
    xaxis = dict(title = 'Timestamp'),
    yaxis = dict(title = 'Price')
)

# Create the figure and add the traces
fig = go.Figure(data=[train_trace, test_trace], layout=layout)

# Show the figure
fig.show()

In [14]:
# set the train and test data and print the dimensions of it
train = data.copy()[(data.index >= train_start_date) & (data.index < test_start_date)][['Price']]
test = data.copy()[data.index >= test_start_date][['Price']]

print('Training data shape: ', train.shape)
print('Test data shape: ', test.shape)

Training data shape:  (204, 1)
Test data shape:  (48, 1)


In [15]:
# Prepare data for training
scaler = MinMaxScaler()
scaled_train = train.copy()
scaled_test = test.copy()
scaled_train['Price'] = scaler.fit_transform(scaled_train[['Price']])
scaled_test['Price'] = scaler.transform(scaled_test[['Price']])  # Scale the test data
print(f'Scaled Training Set: {scaled_train.shape}\nScaled Testing Set {scaled_test.shape}')

Scaled Training Set: (204, 1)
Scaled Testing Set (48, 1)


In [16]:
# Convert to numpy arrays
scaled_train_data = scaled_train.values
scaled_test_data = scaled_test.values

In [17]:
timesteps = 3

In [18]:
scaled_train_data_timesteps=np.array([[j for j in scaled_train_data[i:i+timesteps]] for i in range(0,len(scaled_train_data)-timesteps+1)])[:,:,0]
scaled_train_data_timesteps.shape

(202, 3)

In [19]:
scaled_train_data

array([[0.        ],
       [0.00200296],
       [0.0354437 ],
       [0.05286075],
       [0.04989985],
       [0.03962379],
       [0.04972568],
       [0.05486371],
       [0.07280327],
       [0.06574937],
       [0.03953671],
       [0.06522686],
       [0.08900113],
       [0.10798572],
       [0.08264391],
       [0.04206218],
       [0.04711312],
       [0.06270138],
       [0.07184534],
       [0.08220848],
       [0.05939214],
       [0.07846382],
       [0.07724462],
       [0.08377602],
       [0.09231037],
       [0.0903945 ],
       [0.11120787],
       [0.0874336 ],
       [0.14952539],
       [0.13550466],
       [0.15422799],
       [0.1895846 ],
       [0.1990769 ],
       [0.25620482],
       [0.21414265],
       [0.18009231],
       [0.21701646],
       [0.22546373],
       [0.29016807],
       [0.28964556],
       [0.26029783],
       [0.31037185],
       [0.33066272],
       [0.38213011],
       [0.38108508],
       [0.34381259],
       [0.31568405],
       [0.327

In [12]:
scaled_test_data_timesteps=np.array([[j for j in scaled_test_data[i:i+timesteps]] for i in range(0,len(scaled_test_data)-timesteps+1)])[:,:,0]
scaled_test_data_timesteps.shape

(46, 3)

In [13]:
X_train, y_train = scaled_train_data_timesteps[:,:timesteps-1],scaled_train_data_timesteps[:,[timesteps-1]]
X_test, y_test = scaled_test_data_timesteps[:,:timesteps-1],scaled_test_data_timesteps[:,[timesteps-1]]

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(202, 2) (202, 1)
(46, 2) (46, 1)


In [14]:
SVR??

Init signature:
SVR(
    *,
    kernel='rbf',
    degree=3,
    gamma='scale',
    coef0=0.0,
    tol=0.001,
    C=1.0,
    epsilon=0.1,
    shrinking=True,
    cache_size=200,
    verbose=False,
    max_iter=-1,
)
Source:        
class SVR(RegressorMixin, BaseLibSVM):
    """Epsilon-Support Vector Regression.

    The free parameters in the model are C and epsilon.

    The implementation is based on libsvm. The fit time complexity
    is more than quadratic with the number of samples which makes it hard
    to scale to datasets with more than a couple of 10000 samples. For large
    datasets consider using :class:`~sklearn.svm.LinearSVR` or
    :class:`~sklearn.linear_model.SGDRegressor` instead, possibly after a
    :class:`~sklearn.kernel_approximation.Nystroem` transformer or
    other :ref:`kernel_approximation`.

    Read more in the :ref:`User Guide <svm_regression>`.

    Parameters
    ----------
    kernel : {'linear', 'poly', 'rbf', 'sigmoid', 'precomputed'} or callable,  \

In [15]:
svr = SVR()

In [16]:
param_distributions = {
    'C': uniform(1, 10),  # Regularization parameter
    'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5],  # Margin of tolerance
    'gamma': reciprocal(0.001, 0.1),  # Kernel coefficient
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']  # Kernel type
}

# Perform randomized search
rnd_search_cv = RandomizedSearchCV(svr, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train, y_train)

# Print the best parameters
print(f"Best parameters: {rnd_search_cv.best_params_}")

# Fit and predict with the best parameters
best_svr = rnd_search_cv.best_estimator_
best_svr.fit(X_train, y_train)
y_pred = best_svr.predict(X_test)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END C=7.8236175581616285, epsilon=0.2, gamma=0.020230350001094514, kernel=sigmoid; total time=   0.0s
[CV] END C=7.8236175581616285, epsilon=0.2, gamma=0.020230350001094514, kernel=sigmoid; total time=   0.0s
[CV] END C=7.8236175581616285, epsilon=0.2, gamma=0.020230350001094514, kernel=sigmoid; total time=   0.0s
[CV] END C=9.182457365405083, epsilon=0.1, gamma=0.005308267005572166, kernel=poly; total time=   0.0s
[CV] END C=9.182457365405083, epsilon=0.1, gamma=0.005308267005572166, kernel=poly; total time=   0.0s
[CV] END C=9.182457365405083, epsilon=0.1, gamma=0.005308267005572166, kernel=poly; total time=   0.0s
[CV] END C=9.539882358306665, epsilon=0.5, gamma=0.006475954213686121, kernel=poly; total time=   0.0s
[CV] END C=9.539882358306665, epsilon=0.5, gamma=0.006475954213686121, kernel=poly; total time=   0.0s
[CV] END C=9.539882358306665, epsilon=0.5, gamma=0.006475954213686121, kernel=poly; total time=   0.0s


d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\utils\validation.py:1229: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\utils\validation.py:1229: D

In [17]:
# Inverse transform the scaled data
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))
y_pred_inv = scaler.inverse_transform(y_pred.reshape(-1, 1))
y_pred_inv

array([[ 68.13558008],
       [ 69.78278687],
       [ 74.29799853],
       [ 71.37131018],
       [ 63.48222466],
       [ 66.94233934],
       [ 61.20395041],
       [ 65.83349564],
       [ 61.95200828],
       [ 66.2663669 ],
       [ 68.16450571],
       [ 65.71704064],
       [ 56.85148688],
       [ 34.33440989],
       [ 32.69542968],
       [ 41.07561362],
       [ 49.35242782],
       [ 49.60260705],
       [ 50.92715277],
       [ 46.76558968],
       [ 47.12488809],
       [ 50.27366259],
       [ 56.64411039],
       [ 60.61953177],
       [ 67.0417839 ],
       [ 69.06179065],
       [ 67.41672547],
       [ 71.01163452],
       [ 75.84762375],
       [ 75.43303196],
       [ 70.82941833],
       [ 76.95252208],
       [ 85.79965807],
       [ 79.7300638 ],
       [ 73.81580209],
       [ 87.80245583],
       [ 94.69302762],
       [112.92998461],
       [ 99.84432393],
       [108.40297757],
       [113.18210725],
       [ 97.67721221],
       [ 92.8044857 ],
       [ 86

In [18]:
# plot with plotly
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=test.index[timesteps-1:], y=y_test.flatten(), mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test.index[timesteps-1:], y=y_pred, mode='lines', name='Predicted'))
fig.update_layout(title='Actual vs Predicted', xaxis_title='Date', yaxis_title='Price')
fig.show()


In [20]:
# Random Forest Regressor
rf = RandomForestRegressor()

# Define the hyperparameters for the Random Forest
param_distributions_rf = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(1, 10),
    'min_samples_split': uniform(0.01, 0.2),
    'min_samples_leaf': uniform(0.01, 0.2),
}

# Perform randomized search for Random Forest
rnd_search_cv_rf = RandomizedSearchCV(rf, param_distributions_rf, n_iter=10, verbose=2, cv=3)
rnd_search_cv_rf.fit(X_train, y_train)

# Print the best parameters for Random Forest
print(f"Random Forest - Best parameters: {rnd_search_cv_rf.best_params_}")

# Fit and predict with the best parameters for Random Forest
best_rf = rnd_search_cv_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)

# Calculate the MAE and MAPE for Random Forest
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf)
print(f"Random Forest - MAE: {mae_rf}\nRandom Forest - MAPE: {mape_rf}")

fig = go.Figure()
fig.add_trace(go.Scatter(x=test.index[timesteps-1:], y=y_test.flatten(), mode='lines', name='Actual'))
fig.add_trace(go.Scatter(x=test.index[timesteps-1:], y=y_pred_rf, mode='lines', name='Random Forest'))

Fitting 3 folds for each of 10 candidates, totalling 30 fits


d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\base.py:1351: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



[CV] END max_depth=4, min_samples_leaf=0.18250330557557132, min_samples_split=0.20276165704353097, n_estimators=342; total time=   0.1s
[CV] END max_depth=4, min_samples_leaf=0.18250330557557132, min_samples_split=0.20276165704353097, n_estimators=342; total time=   0.1s


d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\base.py:1351: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\base.py:1351: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



[CV] END max_depth=4, min_samples_leaf=0.18250330557557132, min_samples_split=0.20276165704353097, n_estimators=342; total time=   0.1s
[CV] END max_depth=2, min_samples_leaf=0.11680198170057049, min_samples_split=0.1536877323079994, n_estimators=300; total time=   0.1s


...

[CV] END max_depth=4, min_samples_leaf=0.02883326916215624, min_samples_split=0.04733217012358481, n_estimators=453; total time=   0.2s
[CV] END max_depth=4, min_samples_leaf=0.14293873028005252, min_samples_split=0.13793251494566378, n_estimators=251; total time=   0.0s


d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\base.py:1351: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\base.py:1351: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



[CV] END max_depth=4, min_samples_leaf=0.14293873028005252, min_samples_split=0.13793251494566378, n_estimators=251; total time=   0.1s
[CV] END max_depth=4, min_samples_leaf=0.14293873028005252, min_samples_split=0.13793251494566378, n_estimators=251; total time=   0.1s


d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\base.py:1351: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().

d:\DEV WORK\Data Science Library\ML-For-Beginners\.venv\lib\site-packages\sklearn\base.py:1351: DataConversionWarning:

A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().



Random Forest - Best parameters: {'max_depth': 4, 'min_samples_leaf': 0.02883326916215624, 'min_samples_split': 0.04733217012358481, 'n_estimators': 453}
Random Forest - MAE: 0.04533427900328537
Random Forest - MAPE: 0.1604300590671795


In [21]:

# XGBoost Regressor
xgb = XGBRegressor()

# Define the hyperparameters for XGBoost
param_distributions_xgb = {
    'n_estimators': randint(100, 500),
    'max_depth': randint(1, 10),
    'learning_rate': uniform(0.01, 0.6),
    'subsample': uniform(0.5, 0.5),
    'colsample_bytree': uniform(0.5, 0.5),
}

# Perform randomized search for XGBoost
rnd_search_cv_xgb = RandomizedSearchCV(xgb, param_distributions_xgb, n_iter=10, verbose=2, cv=3)
rnd_search_cv_xgb.fit(X_train, y_train)

# Print the best parameters for XGBoost
print(f"XGBoost - Best parameters: {rnd_search_cv_xgb.best_params_}")

# Fit and predict with the best parameters for XGBoost
best_xgb = rnd_search_cv_xgb.best_estimator_
y_pred_xgb = best_xgb.predict(X_test)

# Calculate the MAE and MAPE for XGBoost
mae_xgb = mean_absolute_error(y_test, y_pred_xgb)
mape_xgb = mean_absolute_percentage_error(y_test, y_pred_xgb)
print(f"XGBoost - MAE: {mae_xgb}\nXGBoost - MAPE: {mape_xgb}")

# plot with plotly
# import plotly.graph_objects as go


fig.add_trace(go.Scatter(x=test.index[timesteps-1:], y=y_pred_xgb, mode='lines', name='XGBoost'))
fig.add_trace(go.Scatter(x=test.index[timesteps-1:], y=y_pred, mode='lines', name='SVR'))
fig.update_layout(title='Actual vs Predicted', xaxis_title='Date', yaxis_title='Price')



Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END colsample_bytree=0.5115282805277437, learning_rate=0.21168441779649533, max_depth=6, n_estimators=401, subsample=0.7828145396319661; total time=   0.1s
[CV] END colsample_bytree=0.5115282805277437, learning_rate=0.21168441779649533, max_depth=6, n_estimators=401, subsample=0.7828145396319661; total time=   0.0s
[CV] END colsample_bytree=0.5115282805277437, learning_rate=0.21168441779649533, max_depth=6, n_estimators=401, subsample=0.7828145396319661; total time=   0.0s
[CV] END colsample_bytree=0.7471382535746314, learning_rate=0.23646993140958084, max_depth=4, n_estimators=149, subsample=0.9617475608553886; total time=   0.0s
[CV] END colsample_bytree=0.7471382535746314, learning_rate=0.23646993140958084, max_depth=4, n_estimators=149, subsample=0.9617475608553886; total time=   0.0s
[CV] END colsample_bytree=0.7471382535746314, learning_rate=0.23646993140958084, max_depth=4, n_estimators=149, subsample=0.9617475608